In [1]:
!pip install git+https://github.com/LBNL-ETA/DOPER

  Cloning https://github.com/LBNL-ETA/DOPER to /tmp/pip-req-build-6iitro4b
  Running command git clone --filter=blob:none --quiet https://github.com/LBNL-ETA/DOPER /tmp/pip-req-build-6iitro4b
  Resolved https://github.com/LBNL-ETA/DOPER to commit 139ddd78c856b12385fb9d9b72f533b348006984
  Preparing metadata (setup.py) ... done
  Created wheel for DOPER: filename=DOPER-2.1.1-py3-none-any.whl size=6005959 sha256=f40a71a17c2db8e59dafbf48da1bd68a6479fc78aad64bf3474625bdd79da810
  Stored in directory: /tmp/pip-ephem-wheel-cache-gn8fihde/wheels/61/c9/71/ccfe329fed449d90b13ae642b4131c25473dadb6cf8de18692
Successfully built DOPER

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
from doper import DOPER, get_solver, standard_report
from doper.models.basemodel import base_model
from doper.models.battery import add_battery
# import doper.example as example

In [7]:
from pyomo.environ import Objective, minimize

def control_model(inputs, parameter):
    model = base_model(inputs, parameter)
    model = add_battery(model, inputs, parameter)
    
    def objective_function(model):
        return model.sum_energy_cost * parameter['objective']['weight_energy'] \
               + model.sum_demand_cost * parameter['objective']['weight_demand'] \
               - model.sum_export_revenue * parameter['objective']['weight_export']
    model.objective = Objective(rule=objective_function, sense=minimize, doc='objective function')
    return model

In [8]:
# load generic project parameter from example
parameter = example.default_parameter()

NameError: name 'example' is not defined

In [ ]:
# Define Optimization (Time-series) Input
dataPV 
dataLoad

In [ ]:
# Define the path to the solver executable
solver_path = get_solver('cbc')

# Initialize DOPER
smartDER = DOPER(model=control_model,
                 parameter=parameter,
                 solver_path=solver_path,
                output_list=my_output_list)

# Conduct optimization
res = smartDER.do_optimization(data)

# Get results
duration, objective, df, model, result, termination, parameter = res
print(standard_report(res))

In [ ]:
my_output_list = [
    {
        'data': '{var or param name within pyomo model}',
        'df_label': '{Label for your output column}'
    },
    {
        'data': '{another var or param}',
        'df_label': '{another label for your output column}'
    } 
]


In [10]:
# define preprocessor (inputs/outputs much conform with FMI-MLC, see Test_fmi_gym.ipynb)
class microgrid_preprocessor:
    def __init__(self, fmi_gym_parameter):
        self.fmi_gym_parameter = fmi_gym_parameter

    def do_calc(self, data, init):

        # preprocess inputs for controller: Data Selection: The first 24 rows might represent a specific time period or batch of data that the controller needs to operate on. For example, if each row represents one hour, this could represent a 24-hour period.
        ctrl_inputs = data.iloc[:24]

        # compute controller
        action = controller(ctrl_inputs) # need to define your controller as separate function

        # append action to inputs
        data['P_set_battery'] = action['P_set_battery']
        data['P_set_FC'] = action['P_set_FC']

        return data

# add preprocessor to fmi_gym
parameter['preprocessor'] = microgrid_preprocessor

NameError: name 'parameter' is not defined